In [22]:
import fsspec
from pangeo_forge_recipes import storage
from pangeo_forge_recipes.patterns import ConcatDim, FilePattern
from pangeo_forge_recipes.recipes import XarrayZarrRecipe, setup_logging

setup_logging()

ids = list(range(2000, 2022))
id_concat_dim = ConcatDim('id', ids, nitems_per_file=1)


def make_url(id):
    return f'https://storage.googleapis.com/ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-{id}.nc'  # noqa


pattern = FilePattern(make_url, id_concat_dim)

recipe = XarrayZarrRecipe(
    pattern, inputs_per_chunk=1, target_chunks={'TIME': 100}, subset_inputs={'TIME': 20}
)

In [23]:
storage_config = storage.temporary_storage_config()
storage_config.target = storage.FSSpecTarget(
    fsspec.implementations.local.LocalFileSystem(), '/tmp/mar'
)
storage_config.cache = storage.CacheFSSpecTarget(
    fsspec.implementations.local.LocalFileSystem(), '/tmp/pangeo-forge-recipes/input-cache'
)

In [24]:
recipe.storage_config = storage_config

In [25]:
recipe_pruned = recipe.copy_pruned()
recipe_pruned

XarrayZarrRecipe(file_pattern=<FilePattern {'id': 2}>, storage_config=StorageConfig(target=FSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f667e31cf40>, root_path='/tmp/mar'), cache=CacheFSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f667e31cf40>, root_path='/tmp/pangeo-forge-recipes/input-cache'), metadata=MetadataTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f667e31cf40>, root_path='/tmp/tmpdslcaln5/jmDkDXic')), inputs_per_chunk=1, target_chunks={'TIME': 100}, cache_inputs=True, copy_input_to_local_file=False, consolidate_zarr=True, consolidate_dimension_coordinates=True, xarray_open_kwargs={}, xarray_concat_kwargs={}, delete_input_encoding=True, process_input=None, process_chunk=None, lock_timeout=None, subset_inputs={'TIME': 20}, open_input_with_kerchunk=False)

In [26]:
run_function = recipe_pruned.to_function()
run_function

<function pangeo_forge_recipes.executors.python.FunctionPipelineExecutor.compile.<locals>.function()>

In [27]:
run_function()

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='id', index=0, sequence_len=2, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'https://storage.googleapis.com/ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2000.nc'
pangeo_forge_recipes.storage - INFO - File 'https://storage.googleapis.com/ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2000.nc' is already cached
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='id', index=1, sequence_len=2, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'https://storage.googleapis.com/ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2001.nc'
pangeo_forge_recipes.storage - INFO - File 'https://storage.googleapis.com/ldeo-glaciology/MAR/6.5km/MARv3.12-6_5km-daily-ERA5-2001.nc' is already cached
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Found an existing dataset in target
pangeo_forge

In [28]:
import xarray as xr

In [29]:
ds = xr.open_zarr(recipe_pruned.target_mapper)
ds

<xarray.Dataset>
Dimensions:      (TIME: 366, SECTOR1_1: 1, y: 415, x: 230, ATMLAY3_3: 1,
                  OUTLAY: 18, bnds: 2, ZTQLEV: 4, SECTOR: 2, ZUVLEV: 3)
Coordinates:
  * ATMLAY3_3    (ATMLAY3_3) float32 0.9997
  * OUTLAY       (OUTLAY) float32 0.0 0.05 0.1 0.2 0.3 ... 7.5 10.0 15.0 20.0
  * SECTOR       (SECTOR) float32 1.0 2.0
  * SECTOR1_1    (SECTOR1_1) float32 1.0
  * TIME         (TIME) datetime64[ns] 2000-01-01T12:00:00 ... 2000-12-31T12:...
  * ZTQLEV       (ZTQLEV) float32 2.0 10.0 50.0 100.0
  * ZUVLEV       (ZUVLEV) float32 10.0 50.0 100.0
  * x            (x) float32 -637.0 -630.5 -624.0 -617.5 ... 838.5 845.0 851.5
  * y            (y) float32 -3.351e+03 -3.345e+03 -3.338e+03 ... -666.9 -660.4
Dimensions without coordinates: bnds
Data variables: (12/44)
    AL2          (TIME, SECTOR1_1, y, x) float32 dask.array<chunksize=(20, 1, 415, 230), meta=np.ndarray>
    CD           (TIME, y, x) float32 dask.array<chunksize=(20, 415, 230), meta=np.ndarray>
    CM           (TIME, y, x) float32 dask.array<chunksize=(20, 415, 230), meta=np.ndarray>
    COD          (TIME, y, x) float32 dask.array<chunksize=(20, 415, 230), meta=np.ndarray>
    CU           (TIME, y, x) float32 dask.array<chunksize=(20, 415, 230), meta=np.ndarray>
    DATE         (TIME) float32 dask.array<chunksize=(20,), meta=np.ndarray>
    ...           ...
    ZN4          (TIME, SECTOR1_1, y, x) float32 dask.array<chunksize=(20, 1, 415, 230), meta=np.ndarray>
    ZN5          (TIME, SECTOR1_1, y, x) float32 dask.array<chunksize=(20, 1, 415, 230), meta=np.ndarray>
    ZN6          (TIME, SECTOR1_1, y, x) float32 dask.array<chunksize=(20, 1, 415, 230), meta=np.ndarray>
    ZTQLEV_bnds  (ZTQLEV, bnds) float32 dask.array<chunksize=(4, 2), meta=np.ndarray>
    ZUVLEV_bnds  (ZUVLEV, bnds) float32 dask.array<chunksize=(3, 2), meta=np.ndarray>
    crs          |S1 ...
Attributes:
    CDI:                       Climate Data Interface version 1.9.7.1 (http:/...
    CDO:                       Climate Data Operators version 1.9.7.1 (http:/...
    Conventions:               CF-1.6
    contact:                   xavier.fettweis@uliege.be
    date:                      Thu Oct  7 09:18:53 CEST 2021
    institute:                 University of Liege (Belgium)
    model:                     regional climate model MAR (v3.12.0.3)
    pangeo-forge:inputs_hash:  159664643ee57d710087df3f5b9c6b45c54f49f7601680...
    pangeo-forge:recipe_hash:  92e8f467a225d346e27bb043c0c44718b60f927005ef36...
    pangeo-forge:version:      0.9.1